# Chapter 20. Atribute Descriptors

    Learning about descriptors not only provides to a larger toolset, it creates a deeper understanding of how Python works and an apprciation for the elegance of its design.

    -Raymond Hettinger
    Python core developer and guru

Descriptor are a way of reusing the same access logic in multiple attributes. For example, field types in ORMs such as the Django ORM and SQL Alchemy are descriptors,  managing the flow of data from the fields in a databsae record to Python objects attributes.

A descriptor is a class that implements a protocol consisting of the `__get__`, `__set__`, and `__delete__` methods. The property class implements the full descriptor protocol. As usual with protocols, partial implementation are OK. In fact, most descriptors we see in real code implement only `__get__` and `__set__`.

Descriptor are a distinguishing feature of Python, deployed not only at the application level but also in language infrastructure. Besides properties, other ptyhon featurs that leverage descriptors are methods and the classmethod and staticmethod decorators. Understanding descriptors is key to Python mastery.

## Overriding Versus Nonoverriding Descriptors

Reading an attribute through an instance normally returns the attribute defined in the instance, but if there is no such attribute in the instance, a class attribute will be retrieved. On the other hand, assigning to an attribute in an instance normally creates the attribute in the instance, without affecting the class at all.

Essential classes for this example

In [1]:
class Overriding:
    """data descriptor or enforced descriptor"""
    
    def __get__(self, instance, owner):
        pass
    
    def __set__(self, instance, value):
        pass

In [3]:
class OverridingNoGet:
    """an overriding descriptor without __get__"""
    
    def __set__(self, instance, value):
        pass

In [4]:
class NonOverriding:
    """non-data or shadowable descriptor"""
    
    def __get__(self, instance, owner):
        pass

### Overriding Descriptor

A descriptor that implements the `__set__` method is called an *overriding descriptor*, a descriptor implementing `__set__` will override attempts to assign to instance attributes. Properties are also overriding descriptor: if you don't provide a setter function, the default `__set__` from the property class will raise AttributeError to signal that the attribute is read-only.

### Overriding Descriptor without `__get__``

Overriding descriptor implement both `__set__` and `__get__`, but it's also possible to implement only `__set__`. Reading the descriptor through an instance will return the descriptor object itself because there is not `__get__` to handle that access.

### Nonoverriding Descriptor

If a descriptor does not implement `__set__`, then it's nonoverriding descriptor.

> Python contributors and authors use different tems when discusing these concepts. Overring descriptors are also called data descriptors or enforced descriptors. Nonoverriding descriptor are also known as nondata descriptor or shadowable descriptors.

### Methods Are Descriptors

A function within a class becomes a bound method because all user-defined functions have a `__get__` method, therefore they operate as descriptors when attached to a class.

The bound method object also have a `___call__` method, which handles the actual invocation. This method calls the original function referenced in `__func__`, passing the `__self__` attribute of the method as the first argument. That's how the implicit binding of the conventional self argument works.

## Descriptor Usage Tips

*Use property to Keep It Simple*

    The property built-in actually create overriding descriptors implementing both __set__ and __get__, even if you do not define a setter method. The default __set__ of a property raise AttributeError: can't set attribute, so a property is the easiest way create a read-only attribute, avoiding the issue descriped next.
    
*Read-only descriptor require `__set__`*

    If you use a descriptor class to implement a read-only attribute, you must remember to code with both __get__ and __set__, otherwise setting a namesake attribute on an instance will shadow the descriptor. The __set__ method of a read-only attribute shold just raise AttributeError with a suitable message.
    
*Validation descriptor can work with `__set__` only*
    
    In a descriptor designed only for validation, the __set__ method should check the value argument it gets, and if valid, set it directly in the instance __dict__ using the descriptor instance name as key. That way, reading the attribute with the same name from the instance will be as fast as possible, because it will not require a __get__.
    
*Caching can be done efficiently with __get__ only*

    If you code just __get__ method, you have a nonoverriding descriptor. These are usaful to make some expensive computation and then cache the result by setting an attribute by the same name on the instance.
    
*Nonspecial methods can be shadowed by instance attribute*

    Because functions and methods only implement __get__, they do not handle attempts at setting instance attributes with the same name, so a single assignment like `my_obj.the_method = 7` means that futher access the `the_method` through the instance will retrieve the number 7 - without affecting the class or other instance.